In [78]:
import pandas as pd; pd.set_option('display.max_rows', None)

index=['QB','RB','WR','TE','FLEX','K','DST']
#I don't know what this does right now but its connected to index=index
df=pd.DataFrame({
    'A':[304,498,456,149,200,110,106],
    'B':[314,474,427,180,200,120,116]
},index=index)
#Index is what allows data to be in order

df.head()
df.sum()


A    1823
B    1831
dtype: int64

In [79]:
df['A-B']=df['A']-df['B']
df.head(7)
#goal of fantasy football is to maximize your scoring differential at each position 

,A,B,A-B
QB,304,314,-10
RB,498,474,24
WR,456,427,29
TE,149,180,-31
FLEX,200,200,0
K,110,120,-10
DST,106,116,-10


In [80]:
df.max(axis=1)
#just shows max of each position

QB      314
RB      498
WR      456
TE      180
FLEX    200
K       120
DST     116
dtype: int64

In [81]:
df['VOR']=abs(df['A-B'])
df['VOR'].sort_values(ascending=False)

TE      31
WR      29
RB      24
DST     10
K       10
QB      10
FLEX     0
Name: VOR, dtype: int64

In [82]:
#find a "replacement player" for each position in the draft pool - a player who's projected points represents the average postional value at each position.
#Then, with respect to each player's position (this is important. You want to compare each player's projected points to their position's replacement value), substract out the replacement value you calculated from your replacement player. 
#The value you're left with is each player's value over the typical replacement player, or for short, their value over replacement.

#1 - Look at ADP for the upcoming draft year and look at pick #100.
#2 - Starting from pick 100, go backwards and look for the last WR, RB, QB, and TE picked thus far. These players are your replacement players.


In [83]:
#The beginning 
from bs4 import BeautifulSoup as BS 
import requests

BASE_URL="https://www.fantasypros.com/nfl/adp/ppr-overall.php"

def make_adp_df():
    res = requests.get(BASE_URL)
    if res.ok:
        soup = BS(res.content, 'html.parser')
        table = soup.find('table', {'id': 'data'})
        df = pd.read_html(str(table))[0]
        print('Output after reading the html:\n\n', df.head(), '\n') # so you can see the output at this point
        df = df[['Player Team (Bye)', 'POS', 'AVG']]
        print('Output after filtering:\n\n', df.head(), '\n')
        df['PLAYER'] = df['Player Team (Bye)'].apply(lambda x: ' '.join(x.split()[:-2])) # removing the team and position
        df['POS'] = df['POS'].apply(lambda x: x[:2]) # removing the position rank
        
        df = df[['PLAYER', 'POS', 'AVG']].sort_values(by='AVG')
        
        print('Final output: \n\n', df.head())
        
        return df
        
    else:
        print('oops, something didn\'t work right', res.status_code)
        
df = make_adp_df()
#We wrote a little function called make_adp_df that makes a request to the URL stored in the BASE_URL variable
#If you inspect element on the page, you'll find the data we need is hidden in a table tag with an id of 'data'. 
#If you want to use a different format, go to that URL and toggle the drop down list to your league format.
#The URL will change, and that will be the URL you will use in your function.

replacement_players = {
    'RB': '',
    'WR': '',
    'TE': '',
    'QB': ''
    
}

for _, row in df[:100].iterrows():
    position = row['POS']
    player = row['PLAYER']
    replacement_players[position] = player


Output after reading the html:

    Rank             Player Team (Bye)  POS  ESPN  MFL  RTSports  Fantrax  AVG
0     1  Christian McCaffrey CAR (13)  RB1   1.0  NaN       1.0      1.0  1.0
1     2       Saquon Barkley NYG (11)  RB2   2.0  NaN       2.0      2.0  2.0
2     3      Ezekiel Elliott DAL (10)  RB3   3.0  NaN       3.0      4.0  3.3
3     4         Michael Thomas NO (6)  WR1   5.0  NaN       5.0      3.0  4.3
4     5           Alvin Kamara NO (6)  RB4   4.0  NaN       4.0      6.0  4.7 

Output after filtering:

               Player Team (Bye)  POS  AVG
0  Christian McCaffrey CAR (13)  RB1  1.0
1       Saquon Barkley NYG (11)  RB2  2.0
2      Ezekiel Elliott DAL (10)  RB3  3.3
3         Michael Thomas NO (6)  WR1  4.3
4           Alvin Kamara NO (6)  RB4  4.7 

Final output: 

                 PLAYER POS  AVG
0  Christian McCaffrey  RB  1.0
1       Saquon Barkley  RB  2.0
2      Ezekiel Elliott  RB  3.3
3       Michael Thomas  WR  4.3
4         Alvin Kamara  RB  4.7


In [84]:
replacement_players = {
    'RB': '',
    'WR': '',
    'TE': '',
    'QB': ''
    
}

for _, row in df[:100].iterrows():
    position = row['POS']
    player = row['PLAYER']
    replacement_players[position] = player




In [85]:
# each position has a different associated URL. We'll create a string format here and loop through the possible positions
BASE_URL = 'https://www.fantasypros.com/nfl/projections/{position}.php?week=draft'

def make_projection_df():
    
    # we are going to concatenate our individual position dfs into this larger final_df
    final_df = pd.DataFrame()
    
    #url has positions in lower case
    for position in ['rb', 'qb', 'te', 'wr',]:
        
        res = requests.get(BASE_URL.format(position=position)) # format our url with the position
        if res.ok:
            soup = BS(res.content, 'html.parser')
            table = soup.find('table', {'id': 'data'})
            df = pd.read_html(str(table))[0]
            
            df.columns = df.columns.droplevel(level=0) # our data has a multi-level column index. The first column level is useless so let's drop it.
            df['PLAYER'] = df['Player'].apply(lambda x: ' '.join(x.split()[:-1])) # fixing player name to not include team
            # if you're not doing PPR, don't include this. If you're doing Half PPR,
            # multiply receptions * 1/2
            if 'REC' in df.columns:
                df['FPTS'] = df['FPTS'] + df['REC'] # add receptions if they're in there. 
            
            df['POS'] = position.upper() # add a position column
            
            df = df[['PLAYER', 'POS', 'FPTS']]
            final_df = pd.concat([final_df, df]) # iteratively add to our final_df
        else:
            print('oops something didn\'t work right', res.status_code)
            return
    
    final_df = final_df.sort_values(by='FPTS', ascending=False) # sort df in descending order on FPTS column
    
    return final_df
            
df = make_projection_df()
df.head()

,PLAYER,POS,FPTS
0,Christian McCaffrey,RB,366.9
0,Lamar Jackson,QB,355.2
1,Patrick Mahomes,QB,341.9
0,Michael Thomas,WR,326.3
2,Dak Prescott,QB,314.3


In [86]:
replacement_values = {
    'RB': 0,
    'WR': 0,
    'QB': 0,
    'TE': 0
}

for position, player in replacement_players.items():
    replacement_values[position] = df.loc[df['PLAYER'] == player].values[0, -1]
    
replacement_values

{'QB': 270.1, 'RB': 128.6, 'TE': 150.7, 'WR': 182.0}

In [87]:
df['VOR'] = df.apply(
    lambda row: row['FPTS'] - replacement_values.get(row['POS']), axis=1
)
df.head()

,PLAYER,POS,FPTS,VOR
0,Christian McCaffrey,RB,366.9,238.3
0,Lamar Jackson,QB,355.2,85.1
1,Patrick Mahomes,QB,341.9,71.8
0,Michael Thomas,WR,326.3,144.3
2,Dak Prescott,QB,314.3,44.2


In [88]:
df = df.sort_values(by='VOR', ascending=False)
df['VALUERANK'] = df['VOR'].rank(ascending=False)
df.head(200)

,PLAYER,POS,FPTS,VOR,VALUERANK
0,Christian McCaffrey,RB,366.9,238.3,1.0
1,Saquon Barkley,RB,306.5,177.9,2.0
2,Ezekiel Elliott,RB,299.4,170.8,3.0
4,Dalvin Cook,RB,290.7,162.1,4.0
5,Alvin Kamara,RB,288.8,160.2,5.0
0,Michael Thomas,WR,326.3,144.3,6.0
3,Derrick Henry,RB,257.3,128.7,7.0
14,Austin Ekeler,RB,250.0,121.4,8.0
7,Clyde Edwards-Helaire,RB,249.0,120.4,9.0
12,Kenyan Drake,RB,246.7,118.1,10.0
